In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
import math

from dateutil.relativedelta import *
from datetime import date
from datetime import datetime

def formatarStr (DataFrame, columns):
    for col in columns:
        DataFrame[col] = DataFrame[col].str.strip()
        
def calcularSoma(cboStr):
    try: 
        df_soma = df_result.query("cbo == \""+cboStr+"\"" )
        total= df_soma['qtde'].sum()    
        return total
    except (RuntimeError, TypeError, NameError, ValueError):
        return np.nan

In [2]:
#ISO-8859-1
nomeArquivoOrigemCBO = 'datasets/origem/MPS/ACT09.csv'
nomeArquivoResultadoCBOTotal = 'datasets/result/dados_CBO_MPS_t.csv'
nomeArquivoResultadoCBO = 'datasets/result/dados_CBO_MPS.csv'

columns_types = {'ano': str, 'cbo': str, 'motivo"': str, 'qtde': str, 'situacao': str}

nomes  = ['ano', 'cbo', 'motivo', 'qtde', 'situacao']

dados_originais = pd.read_csv(nomeArquivoOrigemCBO, sep = ',', header=0, names=nomes, dtype=columns_types, encoding='ISO-8859-1')

df_original = pd.DataFrame(data = dados_originais, columns=nomes)
df_original.shape[0] #1984

1984

In [3]:
df_original.head()

,ano,cbo,motivo,qtde,situacao
0,2004,Membros das Forças Armadas,Típico-Com Cat,310,Com Cat Registrada
1,2004,Membros das Forças Armadas,Trajeto-Com Cat,69,Com Cat Registrada
2,2004,Membros das Forças Armadas,Doença do Trabalho-Com Cat,15,Com Cat Registrada
3,2004,Policiais Militares,Típico-Com Cat,286,Com Cat Registrada
4,2004,Policiais Militares,Trajeto-Com Cat,39,Com Cat Registrada


In [4]:
df_original.drop(['motivo'], axis=1, inplace = True)
formatarStr(df_original, ['ano', 'cbo', 'qtde', 'situacao'])

selecaoRegistrados = df_original['situacao'] == 'Com Cat Registrada'
df_original = df_original[selecaoRegistrados]
df_original.shape[0] #1939

1939

In [5]:
df_original.drop(['situacao'], axis=1, inplace = True)
df_original.dropna(subset = ['qtde'], inplace = True)
df_original.dropna(subset = ['ano'], inplace = True)

df_original['ano'] = df_original['ano'].apply(pd.to_numeric)
df_original['qtde'] = df_original['qtde'].apply(pd.to_numeric)

selecao2016 = df_original['ano'] >= 2016
df_original = df_original[selecao2016]

#excluir os registros cujo cbo foi preenchido com 'Ignorado'
selecaoIgnorado = df_original['cbo'] != 'Ignorado'
df_original = df_original[selecaoIgnorado]
df_original.shape[0] #404

404

In [6]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404 entries, 1563 to 1978
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ano     404 non-null    int64 
 1   cbo     404 non-null    object
 2   qtde    404 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 12.6+ KB


In [7]:
df_result = pd.DataFrame(df_original.groupby(['cbo', 'ano'], as_index=False)['qtde'].sum())
df_result.head(30)

,cbo,ano,qtde
0,Bombeiros Militares,2016,2
1,Bombeiros Militares,2017,1
2,Bombeiros Militares,2018,4
3,"Comunicadores, Artistas e Religiosos",2016,621
4,"Comunicadores, Artistas e Religiosos",2017,592
5,"Comunicadores, Artistas e Religiosos",2018,619
6,Diretores e Gerentes em Empresa de Serviços D,2016,185
7,Diretores e Gerentes em Empresa de Serviços D,2017,172
8,Diretores e Gerentes em Empresa de Serviços D,2018,230
9,Dirigentes de Empresas e Organizações (Exceto,2016,183


In [8]:
df_result.shape[0] #140

140

In [9]:
df_resultGroup = pd.DataFrame({'cbo': df_result['cbo']})
df_resultGroup.drop_duplicates(subset ="cbo", keep = 'first', inplace = True)
df_resultGroup['total'] = df_resultGroup['cbo'].apply(lambda x: calcularSoma(x))
df_resultGroup = df_resultGroup.sort_values(['total'], ascending = False)

df_resultGroup.index = range(df_resultGroup.shape[0]) #0:47
df_resultTop = df_resultGroup[0:10].copy()
df_resultTop.rename(columns={"cbo": "cbo_desc"}, inplace=True)

df_resultTop.head(10)


,cbo_desc,total
0,Trabalhadores dos Serviços,254694
1,Trabalhadores de Funções Transversais,181227
2,Escriturários,127318
3,Técnicos de Nível Médio das Ciências Biológic,116708
4,Trabalhadores da Transformação de Metais e De,84283
5,Trabalhadores da Indústria Extrativa e da Con,83599
6,Vendedores e Prestadores de Serviços do Comér,65409
7,"Trabalhadores da Fabricação de Alimentos, Beb",62361
8,Trabalhadores na Exploração Agropecuária,45904
9,Trabalhadores de Atendimento ao Público,44670


In [10]:
codigosCBO = ['514320', '784205', '411005', '322230', '724315', '717020', '521110', '848510', '621005', '421125']
df_resultTop['cbo'] = codigosCBO
df_resultTop.head(10)

,cbo_desc,total,cbo
0,Trabalhadores dos Serviços,254694,514320
1,Trabalhadores de Funções Transversais,181227,784205
2,Escriturários,127318,411005
3,Técnicos de Nível Médio das Ciências Biológic,116708,322230
4,Trabalhadores da Transformação de Metais e De,84283,724315
5,Trabalhadores da Indústria Extrativa e da Con,83599,717020
6,Vendedores e Prestadores de Serviços do Comér,65409,521110
7,"Trabalhadores da Fabricação de Alimentos, Beb",62361,848510
8,Trabalhadores na Exploração Agropecuária,45904,621005
9,Trabalhadores de Atendimento ao Público,44670,421125


In [11]:
df_resultTop.to_csv(nomeArquivoResultadoCBOTotal, sep = ';', encoding='UTF-8')

In [12]:
df_result.to_csv(nomeArquivoResultadoCBO, sep = ';', encoding='UTF-8')